In [5]:
import pandas as pd

# Read the CSV file with the specified encoding
df = pd.read_csv("Stores.csv", encoding='ISO-8859-1')

# Display the data types of the columns
print(df.dtypes)
#Display count of null values
print(df.isnull().sum())

date_formats = ['%d-%m-%Y', '%m/%d/%Y'] #date format available in my file

def parse_dates(date_str): 
    for fmt in date_formats:
        try:
            return pd.to_datetime(date_str, format=fmt) 
        except ValueError:
            continue
    return pd.NaT #print not a time

# Apply the parsing function to the 'date' column
df['Open Date'] = df['Open Date'].apply(parse_dates)

# Convert to the desired format
df['Open Date'] = df['Open Date'].dt.strftime('%d-%m-%Y')


# Fill missing values in 'Square Meters' with 0
df['Square Meters'].fillna(0, inplace=True)

# Display the number of missing values in each column
print(df.isnull().sum())


# Save the cleaned dataset to a new CSV file
df.to_csv('Cleaned_Stores.csv', index=False)

# Display the cleaned dataframe
print(df.tail(10))
print(df.dtypes)


StoreKey           int64
Country           object
State             object
Square Meters    float64
Open Date         object
dtype: object
StoreKey         0
Country          0
State            0
Square Meters    1
Open Date        0
dtype: int64
StoreKey         0
Country          0
State            0
Square Meters    0
Open Date        0
dtype: int64
    StoreKey        Country           State  Square Meters   Open Date
57        58  United States    North Dakota         1330.0  08-07-2007
58        59  United States          Oregon         2000.0  08-08-2012
59        60  United States    Rhode Island         1260.0  04-04-2005
60        61  United States  South Carolina         2000.0  15-12-2012
61        62  United States    South Dakota         1120.0  03-06-2018
62        63  United States            Utah         2000.0  06-03-2008
63        64  United States   Washington DC         1330.0  01-01-2010
64        65  United States   West Virginia         1785.0  01-01-2012
65    

In [22]:
import pandas as pd

# Assuming df is your DataFrame
df['Open Date'] = pd.to_datetime(df['Open Date'], format='%d-%m-%Y')

df.dtypes

StoreKey                  int64
Country                  object
State                    object
Square Meters           float64
Open Date        datetime64[ns]
dtype: object

In [1]:
import pymysql
import pandas as pd

# Connect to MySQL
myconnection = pymysql.connect(host='127.0.0.1', user='root', passwd='password', database="project")

# Load CSV data
df = pd.read_csv("Cleaned_Stores.csv")
df = df.dropna()  # Drop null values

# Replace data types from pandas to MySQL
a = ",".join(f"`{i}` {j}" for i, j in zip(df.columns, df.dtypes)).replace("float64", "FLOAT").replace("object", "TEXT").replace("int64", "INT").replace("datetime64[ns]","DATE")

# Use a valid table name
table_name = "Stores"

# Create the table
create_table_query = f"CREATE TABLE `{table_name}` ({a})"
with myconnection.cursor() as cursor:
    cursor.execute(create_table_query)

# Insert data into the table
for i in range(len(df)):
    insert_query = f"INSERT INTO `{table_name}` VALUES {tuple(df.iloc[i])}"
    with myconnection.cursor() as cursor:
        cursor.execute(insert_query)
    myconnection.commit()

myconnection.close()
